# Homework	#2

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb

In [2]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=False,
    help="True or False (default: False)"
)

parser.add_argument(
    "-b", "--batch_size", type=int, default=64,
    help="Batch size (int, default: 64)"
)

parser.add_argument(
    "-e", "--epochs", type=int, default=1000,
    help="Number of training epochs (int, default: 1000)"
)

args = parser.parse_args(args=[])

In [27]:
import os
import sys

# 경로 설정
DATA_PATH =  os.path.abspath(os.path.join(os.pardir, "_03_homeworks", "homework_2"))  
print("DATA_PATH:", DATA_PATH)

sys.path.append(DATA_PATH)

from titanic_dataset import TitanicTestDataset, get_preprocessed_dataset

DATA_PATH: C:\Users\sjw99\git\link_dl\_03_homeworks\homework_2


In [28]:
# 데이터 로드 함수
def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(len(train_dataset), len(validation_dataset), len(test_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    
  return train_data_loader, validation_data_loader, test_data_loader

# [요구사항 1] titanic 딥러닝 모델 기본 훈련

In [81]:
# 모델 정의
# in_features=10, output=1, 10->32->16->1로 구성
class MyTitanicModel(nn.Module):
  def __init__(self, n_input, n_output, activation_fn):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      activation_fn(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      activation_fn(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델의 뉴런 조합에 대한 고찰
모델의 각 은닉층(hidden layer)에 대한 뉴런 수를 조정하기 위해  
Layer1, Layer2에 대해 (64, 32), (64, 16), (64, 8), (32, 16), (32, 8)  
다섯 가지 조합으로 실험을 수행하였다.  
그 결과, (32, 16) 조합에서 Validation Loss가 가장 낮고 학습 안정성도 가장 우수하게 나타났다.  
이는 첫 번째 층의 뉴런 수가 64일 경우 모델의 표현력이 과도하게 커져 과적합(overfitting) 이 발생했고,  
반대로 두 번째 층의 뉴런 수가 8일 경우 표현력이 부족해 underfitting 경향을 보였기 때문이다.  
따라서 (32, 16) 조합이 모델 복잡도와 일반화 성능 간의 균형을 가장 잘 유지하는 최적 구조로 판단된다.  

In [83]:
# Optimizer 및 모델 생성 함수
def get_model_and_optimizer(activation_fn=nn.ReLU):
    my_model = MyTitanicModel(n_input=10, n_output=1, activation_fn=activation_fn) # 요구사항2에서 활성화 함수를 조정해가며 쓸수 있도록 수정함
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)
    return my_model, optimizer

In [85]:
# 학습 루프 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader, track_best=False):
  n_epochs = wandb.config.epochs
  loss_fn = nn.BCEWithLogitsLoss() # 이진분류를 위해 BinaryCrossEntropy를 사용
  next_print_epoch = 100
  best_val_loss = float('inf')
    
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target'].unsqueeze(1) # model의 output이 (batch_size,1)형태이므로 2차원 텐서형태로 만들어줌
      output_train = model(input)
      loss = loss_fn(output_train, target.float())
      loss_train += loss.item()
      num_trains += 1

      loss.backward()
      optimizer.step()
      optimizer.zero_grad() # c_model.py의 zero_grad가 loss계산과 backward 중간에 있어 위치를 수정함
        
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target'].unsqueeze(1)
        output_validation = model(input)
        loss = loss_fn(output_validation, target.float())
        loss_validation += loss.item()
        num_validations += 1

    train_loss = loss_train / num_trains
    val_loss = loss_validation / num_validations
      
    wandb.log({
      "Epoch": epoch,
      "Training loss": train_loss,
      "Validation loss": val_loss
    })
      
    # 최고 성능 추적 (track_best=True, 요구사항 2에서 사용)
    if track_best and val_loss < best_val_loss:
        best_val_loss = val_loss
        wandb.run.summary["best_val_loss"] = best_val_loss # 마지막 loss가 아닌 최적의 loss를 저장
            
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

In [46]:
# wandb 설정값
current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

wandb.init(
    mode="online",
    project="titanic_training",
    notes="요구사항1: Titanic 기본모델 학습",
    tags=["titanic", "baseline"],
    name=f"titanic_base_{current_time_str}",
    config={
        "epochs": 1000,
        "batch_size": 64,
        "learning_rate": 1e-3,
        "activation": "ReLU",
        "n_hidden_unit_list": [32, 16],
    },
)

In [47]:
# 데이터 load
train_data_loader, validation_data_loader, _ = get_data()

# 모델 및 옵티마이저 정의 (ReLU 사용)
model, optimizer = get_model_and_optimizer()  

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

In [48]:
# 학습 루프 실행
training_loop(
    model=model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
)

Epoch 100, Training loss 0.5688, Validation loss 0.6669
Epoch 200, Training loss 0.5596, Validation loss 0.6551
Epoch 300, Training loss 0.5761, Validation loss 0.6469
Epoch 400, Training loss 0.5581, Validation loss 0.6414
Epoch 500, Training loss 0.5783, Validation loss 0.6370
Epoch 600, Training loss 0.5496, Validation loss 0.6393
Epoch 700, Training loss 0.5364, Validation loss 0.6346
Epoch 800, Training loss 0.5507, Validation loss 0.6441
Epoch 900, Training loss 0.5389, Validation loss 0.6375
Epoch 1000, Training loss 0.5239, Validation loss 0.6322


In [49]:
print("\nWandB Run URL:", wandb.run.url)
wandb.finish()


WandB Run URL: https://wandb.ai/jwdam-korea-university-of-technology-and-education/titanic_training/runs/pr4cv3uv


Epoch,▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇████
Training loss,█▆▅▅▄▃▄▄▃▃▃▂▃▃▃▃▃▃▃▄▂▂▃▂▃▂▂▁▂▂▃▂▂▂▁▃▁▁▂▂
Validation loss,▇██▇█▇▆▆▅▄▄▄▅▃▄▄▅▄▅▃▃▃▃▃▃▃▃▃▂▂▃▃▂▃▅▂▃█▂▁
Epoch,1000
Training loss,0.52391
Validation loss,0.63216


# [요구사항 2] Activation Function과 Batch Size 변경 및 선택하기

In [50]:
activation_list = ['ReLU', 'Sigmoid', 'ELU', 'LeakyReLU']
batch_sizes = [16, 32, 64, 128]

activation_map = {
    'ReLU' : nn.ReLU, 
    'Sigmoid' : nn.Sigmoid,
    'ELU' : nn.ELU,
    'LeakyReLU' : nn.LeakyReLU
}

In [51]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [52]:
import pandas as pd

results = [] # 조합별 결과를 저장할 list

for act in activation_list:
    for bs in batch_sizes:
        current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

        wandb.init(
            mode="online",
            project="titanic_training",
            notes="요구사항2: Activation & BatchSize 실험",
            tags=["titanic", act, f"batch_{bs}"],
            name=f"titanic_{act}_bs{bs}_{current_time_str}",
            config={
                "epochs": 1000,
                "batch_size": bs,
                "learning_rate": 1e-3,
                "activation": act,
                "n_hidden_unit_list": [32, 16],
            },
        )

        # 데이터 로드 
        train_data_loader, validation_data_loader, _ = get_data()
        activation_fn = activation_map[wandb.config.activation]

        # 모델 + 옵티마이저 준비
        model, optimizer = get_model_and_optimizer(activation_fn)

        # 학습 수행
        print(f"--- {act} / {bs} ---")
        training_loop(model, optimizer, train_data_loader, validation_data_loader, track_best=True)

        # 결과 수집
        train_loss = wandb.run.summary.get("Training loss", None)
        val_loss = wandb.run.summary.get("Validation loss", None)
        best_val = wandb.run.summary.get("best_val_loss", None)
        run_url = wandb.run.url

        results.append({
            "Activation": act,
            "Batch Size": bs,
            "Train Loss": train_loss,
            "Last Val Loss": val_loss,
            "Best Val Loss": best_val,
            "WandB URL": run_url
        })
        
        print(f'\n Finished {act} / Batch={bs}')
        print('WandB URL:', wandb.run.url)
        wandb.finish()

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█
Training loss,██▇█▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▅▅▄▄▄▃▃▃▃▃▃▂▃▂▂▂▁▁
Validation loss,▆▅▅▅▅▅▅▅▅▅▅▄▅▄▄▅▄▄▄▄▄▃▄▃▄▂▂▂▃▁▃▁▂█▄▁▁▂▁▁
Epoch,1000
Training loss,0.45425
Validation loss,0.44515
best_val_loss,0.43651


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
Training loss,████▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▂▁
Validation loss,█▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▅▄▄▄▃▃▄▃▃▄██▃▂▅▇▃▃▁▁▁▂▆▂
Epoch,1000
Training loss,0.44391
Validation loss,0.53991
best_val_loss,0.52294


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

...

Epoch,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█
Training loss,████▇▇▆▆▆▆▅▅▅▅▅▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▂▂▂▁▂
Validation loss,▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▃▂▂▁▃▁█▁▁▂▃▁▁▂▁▇▁▁▁▁
Epoch,1000
Training loss,0.43835
Validation loss,0.54715
best_val_loss,0.47562


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇███
Training loss,▇██▆▇▅▄▆▄▅▆▅▆▄▅▄▆▆▄▄▅▄▄▄▄▄▃▄▅▃▃▄▂▃▃▃▂▁▂▃
Validation loss,█▇███▇▆▆▅▄▅▄▄▄▅▅▄▄▅▆▄▄▅▃▄▃▃▆▃▂█▄▃▅▃▁▃▃▄▁
Epoch,1000
Training loss,0.52952
Validation loss,0.55224
best_val_loss,0.5486


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
Training loss,█▇▅▅▅▃▄▄▃▃▄▃▃▃▃▃▃▂▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁
Validation loss,██▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.58408
Validation loss,0.54946
best_val_loss,0.54813


In [53]:
results_df = pd.DataFrame(results)
display(results_df)

best_comb = results_df.loc[results_df["Best Val Loss"].idxmin()]
print("best combination: ")
print(best_comb)

,Activation,Batch Size,Train Loss,Last Val Loss,Best Val Loss,WandB URL
0,ReLU,16,0.454252,0.445152,0.436510,https://wandb.ai/jwdam-korea-university-of-tec...
1,ReLU,32,0.443906,0.539910,0.522936,https://wandb.ai/jwdam-korea-university-of-tec...
2,ReLU,64,0.530060,0.555124,0.533873,https://wandb.ai/jwdam-korea-university-of-tec...
3,ReLU,128,0.550128,0.589377,0.586827,https://wandb.ai/jwdam-korea-university-of-tec...
4,Sigmoid,16,0.598865,0.591493,0.591493,https://wandb.ai/jwdam-korea-university-of-tec...
5,Sigmoid,32,0.624195,0.596323,0.596323,https://wandb.ai/jwdam-korea-university-of-tec...
6,Sigmoid,64,0.649612,0.658097,0.658097,https://wandb.ai/jwdam-korea-university-of-tec...
7,Sigmoid,128,0.663234,0.650931,0.650931,https://wandb.ai/jwdam-korea-university-of-tec...
8,ELU,16,0.451904,0.400144,0.371075,https://wandb.ai/jwdam-korea-university-of-tec...
9,ELU,32,0.478600,0.568754,0.427884,https://wandb.ai/jwdam-korea-university-of-tec...


best combination: 
Activation                                                     ELU
Batch Size                                                      16
Train Loss                                                0.451904
Last Val Loss                                             0.400144
Best Val Loss                                             0.371075
WandB URL        https://wandb.ai/jwdam-korea-university-of-tec...
Name: 8, dtype: object


# 요구사항2 결론
activation_list = ['ReLU', 'Sigmoid', 'ELU', 'LeakyReLU'],  batch_sizes = [16, 32, 64, 128]에 대해 총 16가지의 조합에 대해 실험한 결과 **ELU/batchsize=16**일때 가장 성능이 좋음을 확인했다.    
또한, 성능을 더욱 올리기 위해 epoch을 조정한 결과 epoch이 1000을 넘어서면 과적합의 경향이 보여 1000이 최적이라 판단했다.  

# [요구사항 3] 테스트 및	submission.csv 생성

In [78]:
best_activation_fn = nn.ELU
best_batch_size = 16

wandb.init(
    mode="disabled",
    project="titanic_training",
    notes="요구사항3: 최적 모델로 테스트 예측 및 submission 생성",
    tags=["titanic", "submission"],
    name="titanic_final",  
    config={
        "epochs": 1000,
        "batch_size": best_batch_size,
        "learning_rate": 1e-3,
        "activation": best_activation_fn,
        "n_hidden_unit_list": [32, 16],
    },
)

In [79]:
train_data_loader, validation_data_loader, test_data_loader = get_data()
model, optimizer = get_model_and_optimizer(best_activation_fn)

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

# submission.csv 구성 위치 고찰

`submission.csv`를 생성하는 시점은 **훈련이 완료된 후**,  
모델이 가장 우수한 성능을 보였던 파라미터 상태(`best_state`)를 불러온 시점이 적절하다.

훈련 과정에서는 매 epoch마다 검증 손실(`val_loss`)을 계산하고,  
이전까지의 최소 검증 손실(`best_val_loss`)보다 낮을 경우 다음과 같이 최고 성능의 모델 가중치를 갱신하도록한다:

```python
if val_loss < best_val_loss:
    best_val_loss = val_loss
    best_state = model.state_dict()

이 구조를 통해 훈련 도중 가장 일반화 성능이 좋은 시점의 모델이 저장하고,
모든 학습이 종료된 후 model.load_state_dict(best_state)로 해당 파라미터를 복원하여
테스트 데이터에 대한 예측(submission.csv 생성) 을 수행한다.

결과적으로, 단순히 마지막 epoch의 모델이 아니라
검증 성능이 가장 우수했던 시점의 모델을 기반으로 예측을 수행하는 것이 더 합리적이라 판단된다.

In [80]:
n_epochs = wandb.config.epochs
loss_fn = nn.BCEWithLogitsLoss()  
next_print_epoch = 100

best_val_loss = float('inf') 
best_state = None
        
for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
        input = train_batch['input']
        target = train_batch['target'].unsqueeze(1)
        output_train = model(input)
        loss = loss_fn(output_train, target.float())
        loss_train += loss.item()
        num_trains += 1
          
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
        for validation_batch in validation_data_loader:
            input = validation_batch['input']
            target = validation_batch['target'].unsqueeze(1)
            output_validation = model(input)
            loss = loss_fn(output_validation, target.float())
            loss_validation += loss.item()
            num_validations += 1      
    
    val_loss = loss_validation / num_validations     
    
    # 최고 성능 파라미터 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = model.state_dict()
                
    if epoch >= next_print_epoch:
        print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
        )
        next_print_epoch += 100

Epoch 100, Training loss 0.5495, Validation loss 0.5769
Epoch 200, Training loss 0.5144, Validation loss 0.5377
Epoch 300, Training loss 0.4841, Validation loss 0.4674
Epoch 400, Training loss 0.4709, Validation loss 0.5062
Epoch 500, Training loss 0.4523, Validation loss 0.5453
Epoch 600, Training loss 0.4415, Validation loss 0.5096
Epoch 700, Training loss 0.4552, Validation loss 0.4333
Epoch 800, Training loss 0.4552, Validation loss 0.4832
Epoch 900, Training loss 0.4483, Validation loss 0.4218
Epoch 1000, Training loss 0.4358, Validation loss 0.4220


In [68]:
# 최고 성능 모델 로드
model.load_state_dict(best_state)

<All keys matched successfully>

In [86]:
# submission 생성
y_preds = []

with torch.no_grad():
    for test_batch in test_data_loader:
        X_test = test_batch['input']
        y_pred = model(X_test)
        y_pred_binary = (y_pred >= 0.5).int().flatten() 
        # 0.5 기준으로 이진 분류 (0 또는 1) 후, flatten()으로 (N,1) → (N,) 형태로 변환하여 리스트 확장에 맞게 1차원 벡터로 정리
        y_preds.extend(y_pred_binary.tolist()) # 배치별 결과를 리스트에 누적 저장

In [71]:
submission = pd.DataFrame({
    'PassengerID' : range(892, 892+len(y_preds)),
    'Survived' : y_preds
})

submission.to_csv('submission.csv', index=False)

In [73]:
import os

if os.path.exists("submission.csv"):
    df_check = pd.read_csv("submission.csv")
    # ID 892~1309,총 418개  
    print(f"submission.csv 생성 완료 — 총 {len(df_check)}행")
else:
    print("submission.csv 생성 실패")

submission.csv 생성 완료 — 총 418행


# [요구사항 4]	submission.csv제출 및 등수확인

![Titanic Leaderboard](https://raw.githubusercontent.com/jwdam/dl_homeworks/main/howeworks2/titanic_ranking.png)

# 숙제 후기

이번 과제에서는 **Kaggle의 Titanic 생존자 예측 문제**를 통해  
**이진 분류(Binary Classification)** 기반의 **FCN(Fully Connected Network)** 모델 구조와 학습 과정을 다뤘다.  
탑승객의 다양한 특성을 입력으로 받아 생존 여부(0 또는 1)를 예측하는 모델을 구현하며,  
신경망이 분류 문제를 해결하는 과정을 직접 확인할 수 있었다.  

**층별 뉴런 수를 다르게 설정한 실험**을 통해,  
복잡한 구조(예: 64–32)는 학습 속도는 빠르지만 과적합이 쉽게 발생했고,  
적절한 크기(예: 32–16)의 네트워크가 오히려 더 좋은 검증 성능을 보였다.  
이를 통해 모델의 복잡도가 항상 성능 향상으로 이어지지 않으며,  
**모델의 단순함과 일반화 성능 사이의 균형이 중요**하다는 점을 깨달을 수 있었다.  

이에 더해 단순히 모델을 구성하는 것에 그치지 않고, **활성화 함수(Activation Function)** 와 **Batch Size** 조합에 따른 성능 차이를 비교하며   
**하이퍼파라미터가 학습 과정에 미치는 영향**을 분석했다.  
특히, 실험에서 ReLU보다 ELU 함수가 더 안정적으로 수렴하는 경향을 보였다.  
그리고 너무 큰 Batch Size는 학습이 빠르지만 일반화 성능이 떨어지는 반면,  
작은 Batch Size는 손실이 천천히 감소하지만 더 안정적인 결과를 보여주었다.  

또한, **WandB**를 이용해 학습 과정을 시각화하면서  
Epoch에 따른 **Training Loss**와 **Validation Loss** 변화를 직관적으로 관찰할 수 있었다.  
이를 통해 학습이 진행될수록 훈련 손실은 꾸준히 감소하지만,  
검증 손실이 일정 시점 이후 다시 증가하는 과적합(Overfitting) 현상을 확인할 수 있었고,  
일정 수준의 Epoch 이후에는 추가 학습이 오히려 성능 저하로 이어질 수 있다는 점을 이해하게 되었다.  

마지막으로 학습 도중 가장 낮은 **검증 손실(Validation Loss)** 을 기록한 시점의 파라미터를 저장하고  
그 상태를 복원해 `submission.csv`를 생성함으로써,  
단순히 마지막 Epoch의 결과가 아닌 **최고 성능 시점의 모델**로 예측을 수행하는 방법의 중요성을 확인했다.  